In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import pylab as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore') 
#para que salga el grafico
#pd.set_option('display.max_rows', None)
from statistics import mean
import pylab as plt
from sklearn.preprocessing import StandardScaler

from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import ExtraTreeClassifier 
from sklearn.model_selection import train_test_split as tts   
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import f1_score as f1  
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import pickle

/Users/javi/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#  PREDICCION PARA ESPAÑA

In [6]:
spanish_hits = pd.read_csv('../Proyecto-FInal-Spotify/CSV_primeros/spanish_hits.csv')

spanish_hits.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration
0,Coldplay,Yellow,0.661,0.429,-7.227,0.00239,0.0281,0.2340,0.285,173.372,266.77
1,Amaral,Cómo hablar,0.775,0.606,-4.752,0.03560,0.0291,0.1110,0.626,117.357,241.27
2,Eminem,The Real Slim Shady,0.661,0.949,-4.244,0.03020,0.0572,0.0454,0.760,104.504,284.20
3,Britney Spears,Oops!...I Did It Again,0.834,0.751,-5.444,0.30000,0.0437,0.3550,0.894,95.053,211.16
4,La Oreja de Van Gogh,La Playa,0.552,0.589,-5.707,0.17500,0.0268,0.1060,0.311,88.535,247.69


In [7]:
spanish_hits['top_hit']=[1 for i in range(len(spanish_hits))]


In [8]:
spanish_hits.shape

(2050, 12)

In [9]:
unpopular_spain = pd.read_csv('../Spoti/unpopular_songs.csv')



In [10]:
unpopular_spain = unpopular_spain.sample(n=2050)

In [11]:
unpopular_spain.drop(['explicit','mode','popularity','key','track_id','instrumentalness'],axis=1,inplace=True)

In [12]:
unpopular_spain=unpopular_spain.iloc[:, [10,9,1,0,2,4,3,5,6,7,8]]


In [13]:
unpopular_spain.rename(columns = {'track_artist':'artist_name', 'track_name':'track_name',
                            'duration_ms':'duration'}, inplace = True)

In [14]:
unpopular_spain.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration
1988,Lonely Drifter Karen,Carousel Horses,0.319,0.662,-8.009,0.451,0.0263,0.266,0.41900,78.821,173120
2888,Ocean Wave Sounds,Sanctuary Shores,0.828,0.118,-31.136,0.819,0.0418,0.198,0.00213,83.157,257082
2526,Spiritual Minds Nature Music,Oceanic Night Crickets Sound,0.839,0.305,-20.081,0.755,0.2090,0.946,0.03780,104.531,103000
4016,Blacc Suhn,Smoke Somethin,0.327,0.748,-11.275,0.073,0.3210,0.271,0.72800,93.929,178040
6636,ewio,RED ALERT,0.737,0.270,-7.874,0.484,0.0797,0.118,0.17800,96.528,267438


In [15]:
unpopular_spain['top_hit']=[0 for i in range(len(unpopular_spain))]


In [16]:
unpopular_spain['duration'] = (unpopular_spain['duration']/1000).round(2)

unpopular_spain.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,top_hit
1988,Lonely Drifter Karen,Carousel Horses,0.319,0.662,-8.009,0.451,0.0263,0.266,0.41900,78.821,173.12,0
2888,Ocean Wave Sounds,Sanctuary Shores,0.828,0.118,-31.136,0.819,0.0418,0.198,0.00213,83.157,257.08,0
2526,Spiritual Minds Nature Music,Oceanic Night Crickets Sound,0.839,0.305,-20.081,0.755,0.2090,0.946,0.03780,104.531,103.00,0
4016,Blacc Suhn,Smoke Somethin,0.327,0.748,-11.275,0.073,0.3210,0.271,0.72800,93.929,178.04,0
6636,ewio,RED ALERT,0.737,0.270,-7.874,0.484,0.0797,0.118,0.17800,96.528,267.44,0


In [17]:
spain_hit_or_not = pd.concat([spanish_hits,unpopular_spain]).reset_index(drop=True)

In [18]:
spain_hit_or_not.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,top_hit
0,Coldplay,Yellow,0.661,0.429,-7.227,0.00239,0.0281,0.2340,0.285,173.372,266.77,1
1,Amaral,Cómo hablar,0.775,0.606,-4.752,0.03560,0.0291,0.1110,0.626,117.357,241.27,1
2,Eminem,The Real Slim Shady,0.661,0.949,-4.244,0.03020,0.0572,0.0454,0.760,104.504,284.20,1
3,Britney Spears,Oops!...I Did It Again,0.834,0.751,-5.444,0.30000,0.0437,0.3550,0.894,95.053,211.16,1
4,La Oreja de Van Gogh,La Playa,0.552,0.589,-5.707,0.17500,0.0268,0.1060,0.311,88.535,247.69,1


In [19]:
spain_hit_or_not.drop_duplicates().shape==spain_hit_or_not.shape    

True

In [20]:
spain_hit_or_not=spain_hit_or_not.drop_duplicates()


In [21]:
spain_hit_or_not.drop_duplicates().shape==spain_hit_or_not.shape    

True

In [22]:
spain_hit_or_not_norm = spain_hit_or_not.copy()

In [23]:
spain_hit_or_not_names = spain_hit_or_not.copy()

In [24]:
spain_hit_or_not_norm.drop(['track_name'],axis=1,inplace=True)

In [25]:
spain_hit_or_not_norm['artist_name']=LabelEncoder().fit_transform(spain_hit_or_not_norm['artist_name'])


In [26]:
scaler=StandardScaler()

In [27]:
spain_hit_or_not_norm[['energy','danceability','loudness','acousticness','speechiness','liveness',\
                  'valence','tempo','duration',
                    'artist_name']] = scaler.fit_transform(spain_hit_or_not_norm[['energy',\
                    'danceability','loudness','acousticness','speechiness','liveness',\
                  'valence','tempo','duration','artist_name']])

In [28]:
spain_hit_or_not_norm.head()

,artist_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,top_hit
0,-1.056048,0.121384,-0.994974,0.240818,-0.940171,-0.613814,0.207525,-0.914583,1.813987,0.427830,1
1,-1.546512,0.604563,-0.002155,0.671915,-0.830393,-0.606007,-0.500913,0.388659,-0.104496,0.195269,1
2,-0.692156,0.121384,1.921782,0.760399,-0.848243,-0.386615,-0.878747,0.900783,-0.544704,0.586792,1
3,-1.275570,0.854629,0.811171,0.551382,0.043603,-0.492017,0.904444,1.412908,-0.868396,-0.079336,1
4,0.053428,-0.340603,-0.097510,0.505573,-0.369595,-0.623964,-0.529712,-0.815215,-1.091634,0.253819,1


In [29]:

X=spain_hit_or_not_norm.drop(columns=['top_hit'])
y=spain_hit_or_not_norm.top_hit

dtr=DTR().fit(X, y)

dict(zip(X.columns, dtr.feature_importances_))

sum(dtr.feature_importances_)

X_norm=StandardScaler().fit_transform(X)
dtr=DTR().fit(X_norm, y)

dict(zip(X.columns, dtr.feature_importances_))

rfr=RFR(n_estimators=2000).fit(X_norm, y)

feat_imp = dict(zip(X.columns, rfr.feature_importances_)) 

In [30]:
feat_imp

{'artist_name': 0.06374302884377796,
 'energy': 0.054420237753882494,
 'danceability': 0.06918615931098346,
 'loudness': 0.38736851770913705,
 'acousticness': 0.0668728288811958,
 'speechiness': 0.09166471099764407,
 'liveness': 0.053043377874977704,
 'valence': 0.05618262714569866,
 'tempo': 0.06098041131683486,
 'duration': 0.09653810016586792}

In [44]:
X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)

In [45]:
linearsvc = LinearSVC()
sgdc = SGDClassifier()
mlpc= MLPClassifier()
percept = Perceptron()
LogReg = LogisticRegression()
LogistReg = LogisticRegressionCV()
svc = SVC()
CCCV = CalibratedClassifierCV()
PAC = PassiveAggressiveClassifier()
LP = LabelPropagation()
LS = LabelSpreading()
rfc = RandomForestClassifier()
GBC = GradientBoostingClassifier()
QDA = QuadraticDiscriminantAnalysis()
HGBT = HistGradientBoostingClassifier()
ridgeCV = RidgeClassifierCV()
ridge = RidgeClassifier()
ABC= AdaBoostClassifier()
ETC = ExtraTreesClassifier()
KNC= KNeighborsClassifier()
BC = BaggingClassifier()
berno = BernoulliNB()
LDA = LinearDiscriminantAnalysis()
gauss =GaussianNB()
nusvc= NuSVC()
DTC = DecisionTreeClassifier()
NC = NearestCentroid()
ETC = ExtraTreeClassifier()

In [46]:
def machine_learning():
    
    modelos = { 'linearsvc':LinearSVC(), 'sgdc': SGDClassifier(),
                'mlpc': MLPClassifier(),
                'percept':Perceptron(),
                'LogReg':LogisticRegression(),
                'LogistReg': LogisticRegressionCV(),
                'svc':SVC(),
                'CCCV' :CalibratedClassifierCV(),
                'PAC' : PassiveAggressiveClassifier(),
                'LP' : LabelPropagation(),
                'LS' : LabelSpreading(),
                'rfc' : RandomForestClassifier(),
                'GBC' : GradientBoostingClassifier(),
                'QDA' : QuadraticDiscriminantAnalysis(),
                'HGBT' : HistGradientBoostingClassifier(),
                'ridgeCV' : RidgeClassifierCV(),
                'ridge' : RidgeClassifier(),
                'ABC': AdaBoostClassifier(),
                'ETC' : ExtraTreesClassifier(),
                'KNC' : KNeighborsClassifier(),
                'BC' : BaggingClassifier(),
                'berno' : BernoulliNB(),
                'LDA' : LinearDiscriminantAnalysis(),
                'gauss' : GaussianNB(),
                'nusvc' : NuSVC(),
                'DTC' : DecisionTreeClassifier(),
                'NC' : NearestCentroid()
              }
    
    nombres = ['linearsvc', 'sgdc', 'mlpc', 'percept',
               'LogReg', 'LogistReg',\
               'svc', 'CCCV', 'PAC',
               'LP', 'LS',
               'rfc', 'GBC', 'QDA', 'HGBT',\
               'ridgeCV', 'ridge', 'ABC', 'ETC',
               'KNC', 'BC', 'berno', 'LDA',\
               'gauss', 'nusvc', 'DTC', 'NC'
              ]
    
    accuracy = []
    #precision = []
    #recall = []
    F1_list = []
    
    resumen_modelos = pd.DataFrame()

    for m in modelos:

        mod = modelos[m]
        mod.fit(X_train, y_train)

        y_pred = mod.predict(X_test)
        accu = acc(y_test, y_pred)
        
       # y_pred = mod.predict(X_test)
        #precis = prec(X_test, y_test)

        #y_pred = mod.predict(X_train) 
        #reca = rec(X_train, y_train)
        
        y_pred = mod.predict(X_test) 
        F1 = f1(y_test, y_pred)

        accuracy.append(accu)
        #precision.append(precis)
        #recall.append(recal)
        F1_list.append(F1)

    resumen_modelos['Modelo'] = nombres
    resumen_modelos['ACC'] = accuracy
    #resumen_modelos['precision'] = precision
    #resumen_modelos['recall'] = recall
    resumen_modelos['F1'] = F1_list
    
    dict_errores = dict(zip(nombres, accuracy))
    best = list(dict_errores.keys())[list(dict_errores.values()).index(max(accuracy))] 
    
    modelo = modelos[best]
    
    modelo.fit(X_train, y_train)
    
    y_pred = modelo.predict(X_test)
    acc_test = modelo.score(X_test, y_test)
    y_pred = modelo.predict(X_train) 
    acc_train = modelo.score(X_train, y_train)
    
    accuracy = max(accuracy)
    #precision = round(precis, 4)
    #recall = round(recal, 4)
    F1 = round(F1, 4)
    
    if acc_train > (1.15*acc_test):
        print(f'Best Model: {best}, acc = {accuracy}, acc_train = {acc_train}, acc_test = {acc_test}, OVERFITING (modifica datos)')
    
    elif acc_train > acc_test:
        print(f'Best Model: {best}, acc = {accuracy}, acc_train = {acc_train}, acc_test = {acc_test}, LO NORMAL')
    
    elif acc_train < acc_test:
        print(f'Best Model: {best}, acc = {accuracy}, acc_train = {acc_train}, acc_test = {acc_test}, UNDERFITING (dame más datos)')
        
    return resumen_modelos

In [47]:
machine_learning()

Best Model: rfc, acc = 0.8195121951219512, acc_train = 1.0, acc_test = 0.8158536585365853, OVERFITING (modifica datos)


,Modelo,ACC,F1
0,linearsvc,0.787805,0.814103
1,sgdc,0.775610,0.788018
2,mlpc,0.803659,0.821310
3,percept,0.686585,0.669241
4,LogReg,0.792683,0.816415
5,LogistReg,0.786585,0.810401
6,svc,0.800000,0.820569
7,CCCV,0.795122,0.818575
8,PAC,0.714634,0.779245
9,LP,0.710976,0.739846


In [48]:
GBC.fit(X_train, y_train)

GradientBoostingClassifier()

In [49]:
y_pred = GBC.predict(X_test)

In [50]:
indices = list(X_test.index)

In [51]:
probab = spain_hit_or_not_norm.loc[indices].reset_index(drop=True)

In [52]:
pred = pd.Series(y_pred) 


In [53]:
pred_test=pd.DataFrame(y_test).reset_index(drop=True)


In [54]:
probab['prediccion'] = pred.round(decimals = 0)

In [55]:
probab['fallo'] = probab['top_hit'] - probab['prediccion']
probab[probab['fallo']!=0]

,artist_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,top_hit,prediccion,fallo
1,-0.468678,0.464695,-1.898046,0.639343,-0.776842,-0.556039,2.465313,-0.413924,1.348434,0.923230,1,0,1
7,-1.321057,1.100457,-0.159211,0.114538,-0.947612,-0.488894,0.098092,1.405264,1.022174,-0.212032,1,0,1
10,0.626954,0.562179,1.102846,0.446700,-0.485290,1.227983,0.754693,1.646039,1.907179,0.329333,1,0,1
11,0.959204,-0.861928,0.115637,0.061761,-0.161343,0.993757,-0.437557,0.155528,0.206112,-0.978480,1,0,1
12,0.211642,1.180987,0.794344,0.847489,0.126243,-0.143020,0.149929,1.657505,0.254746,-0.311623,0,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,-1.400164,0.240059,0.334394,0.504876,-0.930552,-0.566969,0.028976,0.835812,0.334890,0.489116,0,1,-1
803,-0.808839,0.829199,1.540360,0.718247,0.086576,0.462845,-0.569453,-0.085247,-0.630191,0.986250,0,1,-1
809,0.747592,-0.255835,-0.366749,-0.178434,-0.860143,-0.616937,-0.607467,-0.452142,0.201865,0.145747,1,0,1
815,-0.073143,-0.408418,0.934572,-0.181917,-0.409262,-0.292925,0.115371,0.839634,-0.766298,0.373292,1,0,1


# Prediccion para la playlist novedades viernes España

In [56]:
datos_spotipy_week_1_sp = pd.read_csv('../Proyecto-FInal-Spotify/CSV_full/data_week_9DEC_SPAIN.csv')

In [57]:
datos_spotipy_week_1_sp.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration
0,Maria Becerra,LA NENA DE ARGENTINA,0.580,0.531,-4.688,0.0984,0.1880,0.1010,0.887,169.865,151.99
1,Anuel AA,La Máquina,0.775,0.799,-5.869,0.0308,0.0770,0.3090,0.605,95.027,296.91
2,Ptazeta,Mala Mala,0.898,0.797,-4.617,0.0415,0.0414,0.2660,0.667,120.043,152.00
3,KAROL G,La Vida Es Una (From Puss in Boots: The Last W...,0.777,0.819,-7.156,0.0708,0.0513,0.0973,0.625,109.979,189.84
4,Lennis Rodriguez,Infeliz,0.838,0.772,-4.425,0.0261,0.1070,0.0758,0.874,136.166,172.94


In [58]:
datos_spotipy_week_1_sp_names = datos_spotipy_week_1_sp.copy()

In [59]:
datos_spotipy_week_1_sp_norm = datos_spotipy_week_1_sp.copy()

In [60]:
datos_spotipy_week_1_sp_norm['artist_name']=LabelEncoder().fit_transform(datos_spotipy_week_1_sp_norm['artist_name'])


In [61]:
datos_spotipy_week_1_sp_norm[['energy','danceability','loudness','acousticness','speechiness','liveness',\
                  'valence','tempo','duration',
                    'artist_name']] = scaler.fit_transform(datos_spotipy_week_1_sp_norm[['energy',\
                    'danceability','loudness','acousticness','speechiness','liveness',\
                  'valence','tempo','duration','artist_name']])

In [62]:
datos_spotipy_week_1_sp_norm.drop(['track_name'],axis=1,inplace=True)

In [63]:
y_pred = GBC.predict(datos_spotipy_week_1_sp_norm)

In [64]:
pred1 = pd.Series(y_pred) 

In [65]:
datos_spotipy_week_1_sp['prediccion'] = pred1.round(decimals = 0)

In [66]:
datos_spotipy_week_1_sp.prediccion.value_counts()

0    32
1    14
Name: prediccion, dtype: int64

In [67]:
datos_spotipy_week_1_sp.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration,prediccion
0,Maria Becerra,LA NENA DE ARGENTINA,0.580,0.531,-4.688,0.0984,0.1880,0.1010,0.887,169.865,151.99,0
1,Anuel AA,La Máquina,0.775,0.799,-5.869,0.0308,0.0770,0.3090,0.605,95.027,296.91,1
2,Ptazeta,Mala Mala,0.898,0.797,-4.617,0.0415,0.0414,0.2660,0.667,120.043,152.00,0
3,KAROL G,La Vida Es Una (From Puss in Boots: The Last W...,0.777,0.819,-7.156,0.0708,0.0513,0.0973,0.625,109.979,189.84,0
4,Lennis Rodriguez,Infeliz,0.838,0.772,-4.425,0.0261,0.1070,0.0758,0.874,136.166,172.94,1


In [ ]:
#datos_spotipy_week_1_sp_norm.to_csv('../Proyecto-FInal-Spotify/CSV_primeros/datos_spotipy_week_1_sp_norm.csv', index=False)

In [ ]:
pickle.dump(GBC, open('../Proyecto-FInal-Spotify/CSV_primeros/GBC.pkl',
                      'wb'))